be really clear about what's going into repaired tables: 
- repaired
- already OK
- unrepairable

Only ~ 215k repaired `env_package`s ?!
- yeah, there just aren't that many annotated in the biosample metadata


- some of this code should probably be moved back into the package
    - splitting and repiping 

- read in previous SSSOMs and don't re-map? or at least double check?

- don't forget to consult "model" field in biosample table too
    - for scoping, in addition to en_package and taxonomy_id?

- what to do about hits against obsolete classes... does OLS provide a mechanism to avoid?
- be careful with GAZ matches
- avoid IDless, OMIT, NCIT, MICRO, BTO

- is it worth trimming and concatenating "x1", "x2", "x3", where x is a mappable term? numbers may be low.
    - example = XXX

- reconcile "x biome" and "x" mappings
- take note of non-local salvage hits
    - like XXX
    - small/large (freshwater) lake?
    - include native labels in the review frame, in addition to local?
    - should use rdftab, not additional OLS calls
    

- switch from hard-coded `env_package_overrides` to something like SSSOM?
- mappings can come from merging strategies, first pass OLS or salvage OLS
- reflect those methods in SSSOM
- add string distance/confidence
    - multiple columns for multiple methods?

review these SSSOM columns

```
    "subject_category": [biosample_col_to_map] * results_rows,
    "predicate_id": ["skos:relatedMatch"] * results_rows,
    "match_type": ["Lexical"] * results_rows,
    "creator_id": ["https://github.com/turbomam/scoped-mapping"] * results_rows,
    #     "mapping_tool": ["https://www.ebi.ac.uk/ols/docs/api"] * results_rows,
    #     "confidence": 1 - frame_for_sssom["string_dist"],
```

- are there samples with a host_taxid and a (sample) taxonomy_id from "unclassified"?
- start at uncultured or metagenome taxonomy_id?

- what scoping on or mapping of  "gold" or "path"? WON'T FIND THAT IN NCBI BIOSAMPLE


What to save where? 
- new columns in biosample sqlite? 
- delimited file dumps
- move content from original column and insert the new values into that original column?
- pandas clipboard integration handy for review step


In [1]:
import pandas as pd
import re
import requests as requests
import sqlite3
import string
import urllib
import yaml

import scoped_mapping

from datetime import datetime
from pkg_resources import get_distribution, DistributionNotFound
from strsimpy.cosine import Cosine
from xml.etree import ElementTree
from tdda import rexpy

# User-provided data
See repo README for notes on setting up SQLite databases of OBO ontologies with semantic-sql, relation-graph and rdftab

In [2]:
# from https://www.ncbi.nlm.nih.gov/biosample/docs/packages/?format=xml
# see also https://www.ncbi.nlm.nih.gov/biosample/docs/packages/
biosample_packages_file = "../../target/biosample_packages.xml"

# from ftp://ftp.ncbi.nlm.nih.gov//biosample/biosample_set.xml.gz
# via harmonized_table.db.gz
# in https://drive.google.com/drive/u/0/folders/1eL0v0stoduahjDpoDJIk3z2pJBAU4b2Y
biosample_sqlite_file   = "../../target/harmonized-table.db"

# biosample_sqlite_file   = "../../biosample-analysis/target/harmonized-table.db"

# ncbitaxon_sqlite_file   = "../semantic-sql/db/ncbitaxon.db"

# where do we require a single ontology and where can we use multiple?
target_onto_prefix    = "PO"
# 'gaz', 
first_pass_ontologies = ['envo', 'ncbitaxon']

chars_to_whiteout  = "._-"
my_query_fields    = ""  # OLS weighted default
my_row_req         = 3
my_string_dist_arg = 2
my_max_string_dist = 0.1
salvage_max_dist   = 0.2

### What Biosample field should be mapped to ontology classes?

In [3]:
biosample_col_to_map = "env_medium"

### How should the mapping effort be scoped?
Additional taxonomic filters may be applied below

In [4]:
scoping_col   = "env_package_normalized.EnvPackage"
scoping_value = "soil"

### Settings for manual review of OLS search-based results
Results based on merging Biosample annotations to ontology classes by label or embedded ID are not exported for review at this time

In [5]:
# ols_review_file = "../local/ols_review.tsv"
# ols_review_seperator = "\t"

strategy_col = "strategy"
include_col  = "include"

first_pass_include_val  = True
# first_pass_strategy_val = "env_braod_scale vs envo and gaz @ 0.1"
salvage_include_val     = False
# salvage_strategy_val    = "env_braod_scale vs all of ols @ 0.2"

### Settings for SSSOM output

In [6]:
sssom_subject_prefix = biosample_col_to_map + "_" + scoping_col + "_" + scoping_value
sssom_file = sssom_subject_prefix + "_SSSOM.tsv"

In [7]:
# per_biosample_mapping_results_file = "per_biosample_scoped_mapping_results.tsv"

----

## It's nice to see everything accounted for
(In terms of manual mappings for env_package.) But we can prioritize NMDC for now:

- Soil 15,777
- Sediment 7,147
- Plant-associated 3,142

Could some of these "no environmental package" mappings be losing important granularity?

Map `None` and '' to "no environmental package"?

In [8]:
env_package_overrides = {
    "built environment": "built",
    "misc environment": "miscellaneous",
    "missing": "no environmental package",
    "unknown": "no environmental package",
    "default": "no environmental package",
    "unspecified": "no environmental package",
    "not available": "no environmental package",
    "not collected": "no environmental package",
    "miscellaneous natural or artificial environment": "miscellaneous",
    "not applicable": "no environmental package",
    "soil-associated": "soil",
    "soil associated": "soil",
}

In [9]:
biosample_cnx = sqlite3.connect(biosample_sqlite_file)
# ncbitaxon_cnx = sqlite3.connect(ncbitaxon_sqlite_file)
# primary_ontology_sqlite_file = "../semantic-sql/db/" + target_onto_prefix.lower() + ".db"
# primary_ontology_cnx         = sqlite3.connect(primary_ontology_sqlite_file)
first_pass_ontologies.insert(0, target_onto_prefix.lower())
first_pass_ontologies = ','.join(first_pass_ontologies) 

----

## Print a sample of the data we're working with
Specifically, INSDC/NCBI Biosample metadata

In [10]:
q = """
select
    id,
    env_package,
    package,
    package_name,
    host_taxid,
    taxonomy_id,
    env_broad_scale,
    env_local_scale,
    env_medium
    from biosample b
limit 10
"""


biosample_first_ten = pd.read_sql(q, biosample_cnx)
biosample_first_ten

,id,env_package,package,package_name,host_taxid,taxonomy_id,env_broad_scale,env_local_scale,env_medium
0,2,missing,MIGS.ba.5.0,MIGS: cultured bacteria/archaea; version 5.0,9606,445970,terrestrial biome [ENVO:00000446],human-associated habitat [ENVO:00009003],biological product [ENVO:02000043]
1,3,missing,MIGS.ba.5.0,MIGS: cultured bacteria/archaea; version 5.0,9606,445972,terrestrial biome [ENVO:00000446],human-associated habitat [ENVO:00009003],biological product [ENVO:02000043]
2,4,missing,MIGS.ba.5.0,MIGS: cultured bacteria/archaea; version 5.0,9606,449673,terrestrial biome [ENVO:00000446],human-associated habitat [ENVO:00009003],biological product [ENVO:02000043]
3,7,missing,MIGS.ba.5.0,MIGS: cultured bacteria/archaea; version 5.0,9606,445974,terrestrial biome [ENVO:00000446],human-associated habitat [ENVO:00009003],biological product [ENVO:02000043]
4,8,missing,MIGS.ba.5.0,MIGS: cultured bacteria/archaea; version 5.0,9606,411461,terrestrial biome [ENVO:00000446],human-associated habitat [ENVO:00009003],biological product [ENVO:02000043]
5,10,,Generic.1.0,Generic,,451639,,,
6,11,,Generic.1.0,Generic,,451638,,,
7,12,missing,MIGS.ba.5.0,MIGS: cultured bacteria/archaea; version 5.0,9606,451640,terrestrial biome [ENVO:00000446],human-associated habitat [ENVO:00009003],biological product [ENVO:02000043]
8,14,missing,MIGS.ba.5.0,MIGS: cultured bacteria/archaea; version 5.0,9606,470145,terrestrial biome [ENVO:00000446],human-associated habitat [ENVO:00009003],biological product [ENVO:02000043]
9,15,missing,MIGS.ba.5.0,MIGS: cultured bacteria/archaea; version 5.0,9606,471870,terrestrial biome [ENVO:00000446],human-associated habitat [ENVO:00009003],biological product [ENVO:02000043]


## Get the canonical checklist and package terms from NCBI

### Could (should?) also examine mixs.yaml (from where?)

Unfortunately it doesn't do a very good job of differentiating checklists (MIMAG, MIMARKS, etc.) from packages (soil, water, etc.)

_What about .ba, .euk, etc?_

In [11]:
package_dictionary = scoped_mapping.get_package_dictionary(biosample_packages_file)
package_dictionary.to_sql(
    "package_dictionary", biosample_cnx, if_exists="replace", index=False
)
package_dictionary

,Name,DisplayName,ShortName,EnvPackage,EnvPackageDisplay,NotAppropriateFor,Description,Example
0,Generic.1.0,Generic,,,,,Generic,
1,SARS-CoV-2.cl.1.0,SARS-CoV-2: clinical or host-associated; versi...,SARS-CoV-2: clinical or host-associated,,,wgs_single;wgs_batch;wgs_diploid,Use for SARS-CoV-2 samples that are relevant t...,
2,SARS-CoV-2.wwsurv.1.0,SARS-CoV-2: wastewater surveillance; version 1.0,SARS-CoV-2: wastewater surveillance,,,wgs_single;wgs_batch;wgs_diploid,Use for SARS-CoV-2 wastewater surveillance sam...,
3,Pathogen.cl.1.0,Pathogen: clinical or host-associated; version...,Pathogen: clinical or host-associated,,,,Clinical or host-associated pathogen,SAMN02928182
4,Pathogen.env.1.0,Pathogen: environmental/food/other; version 1.0,Pathogen: environmental/food/other,,,,"Environmental, food or other pathogen",SAMN02730065
...,...,...,...,...,...,...,...,...
150,MIUVIG.plant-associated.5.0,"MIUVIG: uncultivated virus genome, plant-assoc...",MIUVIG Uncultivated Virus Genome,plant-associated,plant-associated,wgs_single;wgs_batch;wgs_diploid,Use for uncultivated virus genome identified i...,
151,MIUVIG.sediment.5.0,"MIUVIG: uncultivated virus genome, sediment; v...",MIUVIG Uncultivated Virus Genome,sediment,sediment,wgs_single;wgs_batch;wgs_diploid,Use for uncultivated virus genome identified i...,
152,MIUVIG.soil.5.0,"MIUVIG: uncultivated virus genome, soil; versi...",MIUVIG Uncultivated Virus Genome,soil,soil,wgs_single;wgs_batch;wgs_diploid,Use for uncultivated virus genome identified i...,
153,MIUVIG.wastewater.5.0,"MIUVIG: uncultivated virus genome, wastewater;...",MIUVIG Uncultivated Virus Genome,wastewater,wastewater,wgs_single;wgs_batch;wgs_diploid,Use for uncultivated virus genome identified i...,


## Do the Biosample checklist/package fields match any of the canonical values?

Start by counting the Biosample rows/records. See XXX notes on extracting this *harmonized* database

In [12]:
q = """
select count(*) as biosample_row_count
from biosample b
"""
[biosample_row_count, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx, print_timing=False
)

print(query_duration)
biosample_row_count

0:00:00.111714


,biosample_row_count
0,18068841


## How many of those rows can be inner-joined with the canonical checklists/packages?
Specifically, joining `biosample.package_name` with `package_dictionary.DisplayName`

_Note that indices are built as part of the makefile_


- create index biosample_package_name_idx on biosample(package_name);
- create index package_dictionary_DisplayName_idx on package_dictionary(DisplayName);
- create index biosample_package_idx on biosample(package);
- create index biosample_p_pn_idx on biosample(package, package_name);

In [13]:
q = """
select
    count(*) as canonical_package_name_count
from
    biosample b
inner join package_dictionary pd on
    b.package_name = pd.DisplayName
"""
[canonical_package_name_count, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx, print_timing=True
)

print(query_duration)

canonical_package_name_count

2021-07-04 16:50:47.091119
2021-07-04 16:50:48.073258
0:00:00.982139
0:00:00.982139


,canonical_package_name_count
0,18068841


## Combinations of `package` and `package_name` values in the Biosample dataset

In [14]:
q = """
select
    package,
    package_name,
    count(*) as count
from
    biosample b
group by
    package ,
    package_name
order by
    package ,
    package_name
"""
[package_name_combos, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx, print_timing=True
)

print(query_duration)

package_name_combos

2021-07-04 16:50:48.675359
2021-07-04 16:50:50.711518
0:00:02.036159
0:00:02.036159


,package,package_name,count
0,Beta-lactamase.1.0,Beta-lactamase; version 1.0,630
1,Generic.1.0,Generic,12383104
2,Human.1.0,Human; version 1.0,448733
3,Invertebrate.1.0,Invertebrate; version 1.0,173962
4,MIGS.ba.5.0,MIGS: cultured bacteria/archaea; version 5.0,34259
...,...,...,...
129,Pathogen.cl.1.0,Pathogen: clinical or host-associated; version...,737186
130,Pathogen.env.1.0,Pathogen: environmental/food/other; version 1.0,294681
131,Plant.1.0,Plant; version 1.0,513794
132,SARS-CoV-2.cl.1.0,SARS-CoV-2: clinical or host-associated; versi...,115291


## What about the Biosample `env_package` values?
Are they also a small, highly regular set, like the `package` and `package_name` combinations?

In [15]:
q = """
select
    env_package,
    count(*) as count
from
    biosample b
group by
    env_package
order by
    count(*) desc
"""
[env_package_count, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)

print(query_duration)

env_package_count

0:00:00.942203


,env_package,count
0,,17815491
1,host-associated,69896
2,human-gut,53685
3,water,17554
4,misc environment,14418
...,...,...
100,Unknown,1
101,marine sediment (ENVO:00002113),1
102,ocean[ENVO:00000015],1
103,saliva,1


## `env_package` is a mixture of `ENVO` term ids and strings
Those strings may or may not be the term's labels. There are many redundancies due to small spelling and punctuation variations.

Start by tidying `env_package` etc. values recognized by INSDC

In [16]:
package_dictionary = scoped_mapping.make_tidy_col(
    package_dictionary, "EnvPackage", "eptidy"
)
package_dictionary = scoped_mapping.make_tidy_col(
    package_dictionary, "EnvPackageDisplay", "epdtidy"
)

# update in sqlite
package_dictionary.to_sql(
    "package_dictionary", biosample_cnx, if_exists="replace", index=False
)